In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tensorflow
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow import keras
from keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.18.0


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-03-14 20:25:22--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.06s   

2025-03-14 20:25:22 (6.13 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2025-03-14 20:25:22--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [ ]:
df_train_dataset = pd.read_csv(train_file_path, sep='\t', names=['label', 'text'])
df_test_dataset = pd.read_csv(test_file_path, sep='\t', names=['label', 'text'])
train_dataset = df_train_dataset['text'].to_numpy()
test_dataset = df_test_dataset['text'].to_numpy()
train_label = df_train_dataset['label'].map({'ham':0, 'spam':1}).to_numpy()
test_label = df_test_dataset['label'].map({'ham':0, 'spam':1}).to_numpy()

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

In [ ]:
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset)

In [ ]:
train_dataset = encoder(train_dataset).numpy()
test_dataset = encoder(test_dataset).numpy()

In [ ]:
train_dataset = sequence.pad_sequences(train_dataset, MAXLEN)
test_dataset = sequence.pad_sequences(test_dataset, MAXLEN)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, train_label, epochs=10, batch_size=BATCH_SIZE, validation_split=0.2, verbose=1)

Epoch 1/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 33s 515ms/step - accuracy: 0.8629 - loss: 0.4881 - val_accuracy: 0.9737 - val_loss: 0.1314
Epoch 2/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 41s 516ms/step - accuracy: 0.9765 - loss: 0.0921 - val_accuracy: 0.9797 - val_loss: 0.0671
Epoch 3/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 40s 492ms/step - accuracy: 0.9904 - loss: 0.0462 - val_accuracy: 0.9892 - val_loss: 0.0521
Epoch 4/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 39s 456ms/step - accuracy: 0.9935 - loss: 0.0322 - val_accuracy: 0.9330 - val_loss: 0.2505
Epoch 5/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 42s 472ms/step - accuracy: 0.9926 - loss: 0.0239 - val_accuracy: 0.9904 - val_loss: 0.0519
Epoch 6/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 26s 496ms/step - accuracy: 0.9994 - loss: 0.0055 - val_accuracy: 0.9904 - val_loss: 0.0507
Epoch 7/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 41s 495ms/step - accuracy: 0.9984 - loss: 0.0066 - val_accuracy: 0.9880 - val_loss: 0.0544
Epoch 8/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 40s 467ms/step - accuracy: 0.9991 - loss: 0.0041 - val_accu

In [ ]:
results = model.evaluate(test_dataset, test_label)
print(results)

44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - accuracy: 0.9840 - loss: 0.1141
[0.09807700663805008, 0.983476996421814]


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  pred_text = encoder([pred_text]).numpy()
  pred_text = sequence.pad_sequences(pred_text, MAXLEN)
  pred_text = model.predict(pred_text)[0][0]
  label = 'spam' if pred_text >= 0.5 else 'ham'

  return [float(pred_text), label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
[1.2440137652447447e-05, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
You passed the challenge. Great job!
